# Self study 6
# Group 3

For this assignment the original script has been used. The *weights* parameter were set to 'imagenet' and *None*.

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.xception import Xception

In [2]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train+val', 'train')
     valid_path = os.path.join(base_dir, 'train+val', 'valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
train_gen, val_gen = get_pcam_generators('C:/Users/Daniel/Documents/')
input = Input(input_shape)
# get the pretrained model, cut out the top layer
pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')

#for layer in pretrained.layers[-4]:
#    layer.trainable = False

# Freeze the layers except the last 4 layers
#for i, layer in enumerate(pretrained.layers):
    #layer.trainable = False
#    print(layer.name)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [4]:
layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)
indices = [i for i, s in enumerate(layernames) if 'block14' in s]

for layer in pretrained.layers[min(indices):max(indices)+1]:
    layer.trainable = False
    
 
output = pretrained(input)
output = GlobalAveragePooling2D()(output)
output = Dropout(0.5)(output)
output = Dense(1, activation='sigmoid')(output)

# Check the trainable status of the individual layers
#for layer in pretrained.layers:
#    print(layer, layer.trainable)

model = Model(input, output)

# note the lower lr compared to the cnn example
model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
#
# print a summary of the model on screen
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 20,863,529
Trainable params: 16,060,201
Non-trainable params: 4,803,328
_________________________________________________________________


In [5]:
for i,layer in enumerate(pretrained.layers):
    print(layer.name, layer.trainable)

input_2 False
block1_conv1 True
block1_conv1_bn True
block1_conv1_act True
block1_conv2 True
block1_conv2_bn True
block1_conv2_act True
block2_sepconv1 True
block2_sepconv1_bn True
block2_sepconv2_act True
block2_sepconv2 True
block2_sepconv2_bn True
conv2d_1 True
block2_pool True
batch_normalization_1 True
add_1 True
block3_sepconv1_act True
block3_sepconv1 True
block3_sepconv1_bn True
block3_sepconv2_act True
block3_sepconv2 True
block3_sepconv2_bn True
conv2d_2 True
block3_pool True
batch_normalization_2 True
add_2 True
block4_sepconv1_act True
block4_sepconv1 True
block4_sepconv1_bn True
block4_sepconv2_act True
block4_sepconv2 True
block4_sepconv2_bn True
conv2d_3 True
block4_pool True
batch_normalization_3 True
add_3 True
block5_sepconv1_act True
block5_sepconv1 True
block5_sepconv1_bn True
block5_sepconv2_act True
block5_sepconv2 True
block5_sepconv2_bn True
block5_sepconv3_act True
block5_sepconv3 True
block5_sepconv3_bn True
add_4 True
block6_sepconv1_act True
block6_sepconv1 

In [ ]:
#for i,layer in enumerate(model.layers):
    #print(i,layer.name)

#for layer in model.layers:
#    print(layer.output_shape)
# get the data generators


# save the model and weights
model_name = 'Xception weights ImageNet 14 off'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

model_json = model.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json)


# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]


# train the model, note that we define "mini-epochs"
train_steps = train_gen.n//train_gen.batch_size//20
val_steps = val_gen.n//val_gen.batch_size//20

# since the model is trained for only 10 "mini-epochs", i.e. half of the data is
# not used during training

history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10,
                    callbacks=callbacks_list)

# print a summary of the model on screen
model.summary()

In [ ]:
# Freeze the layers except the last 4 layers
for i,layer in enumerate(pretrained.layers):
    #layer.trainable = False
    print(i, layer.name)
